# Instituto Tecnológico de Costa Rica (ITCR)
## Aprendizaje automático


# Selección de características

---



---

Autores: Saúl Calderón, Juan Esquivel, Luis-Alexander Calvo-Valverde, María Auxiliadora Mora

En general, la gran cantidad de datos existentes, en algunos casos **datos masivos**, por ejemplo, los generados con sensores (ej. para capturar temperatura, calidad del aire, humedad, pH del suelo, entre otros) nos **da la posiblidad de calcular miles de características o «features» posibles** para un conjunto de datos. Sin embargo, crear un **arreglo de características $\vec{x}\in\mathbb{R}^{D}$ con un D muy alto, genera problemas** en cuanto al tiempo de entrenamiento, el cual crece de forma exponencial por la maldición de la dimensionalidad, sumado al riesgo de sobre ajuste al agregar más características al arreglo. Además, algunas de esas característcias pueden ser útiles, otras podrían ser completamente inútiles. Entonces **los mecanismos para selección de características resultan muy necesarios**. 

Los métodos de selección de características tienen como **objetivo reducir las dimensiones de las muestras, minimizando la pérdida de información, y posibilitando la efectividad de los modelos a construir sobre tales datos**. 

A continuación se discuten **tres técnicas básicas de selección de características** que son parte de los métodos supervisados: 

- Métodos de filtrado 
- Métodos de envoltura  
- Métodos empotrados 

La siguiente figura (Brownlee, 2019) muestra las técnicas básicas:

![](../imagenes/metodos_seleccion_caracteristicas.png)
*Figura que muestra algunas técnicas básicas de selección de características y ejemplos de algoritmos como el Recursive feature elimination (RFE).*

## Métodos de filtrado

Los métodos de filtrado **analizan la relación entre las características y la variable de salida o a estimar**. Por ejemplo, si el objetivo del modelo es estimar la temperatura $y$ con variable aleatoria $Y$, y para ello se contempla la construcción de un modelo basado en el vector de características $\vec{x}=\left[x,v\right]$, con $x$ correspondiente a la medición de la humedad y $v$ a la iluminación, y sus variables aleatorias correspondientes $X$ y $V$. A continuación se detalla el método de información mutua.


### Método de Información Mutua

La información mutua entre dos variables aleatorias, por ejemplo Y y X **mide la dependencia entre tales variables aleatorias**. Para realizarlo, si se dispone de la función de densidad de probabilidad de ambas variables aleatorias discretas $p\left[Y=y\right]=p\left[y\right]$ y $p\left[X=x\right]=p\left[x\right]$, la información mutua de ambas variables aleatorias puede expresarse de la siguiente forma: 

$$I\left[X;Y\right]=\sum_{y\in Y}\sum_{x\in X}p\left[x,y\right]\log\left(\frac{p\left[x,y\right]}{p\left[x\right]p\left[y\right]}\right)$$

de forma que: 

1. Si $Y$ y $X$ son independientes, no hay información de $Y$ que se pueda obtener conociendo $X$ ni viceversa, por lo que entonces la Información mutua 
$I\left[X;Y\right]=0$. Esto pues $\log{\left(\frac{p\left[x,y\right]}{p\left[x\right]p\left[y\right]}\right)}=\log{1}=0$

2. Si existe una función determinística que permita calcular $y$ con $x$, entonces la información mutua resulta en $I\left[X;Y\right]=1$. 

Dos beneficios de usar el método de información mutua como selector de características, son:

- Es un método neutral, lo que significa que la solución se puede aplicar a varios tipos de algoritmos de ML.
- La solución es rápida.

### Ejemplo 1
Prueba de tres modelos de clasificación utilizando subconjuntos distintos de características seleccionadas con los resultados del método de información mutua.

Conjuntos de datos:
- 1 Todo el conjunto de datos (nombre con _1).
- 2 Seleccionando las variables con score > 0.2 (_2)
- 3 Seleccionando las variables con score <= 0.2 (_3)

Datos a utilizar: Datos de cáncer de mama de Wisconsin. El conjunto de datos de cáncer de mama es un conjunto de datos utilizado para clasificación binaria clásico.

Composición:
- Classes: 2
- Samples per class: 212(Malignant),357(Benign)
- Samples total: 569
- Dimensionality: 30
- Features: real, positive

Algunas características:
- radius (mean of distances from center to points on the perimeter)        
- texture (standard deviation of gray-scale values)       
- perimeter        
- area        
- smoothness (local variation in radius lengths)        
- compactness (perimeter^2 / area - 1.0)        
- concavity (severity of concave portions of the contour)       
- concave points (number of concave portions of the contour)       
- symmetry
- fractal dimension ("coastline approximation"). Multiple fields.   
- class:
       - WDBC-Malignant                
       - WDBC-Benign


Datos disponibles como parte de los conjuntos de datos del scikit-learn en:
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

El conjunto de datos original y su documentación están disponible en https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

In [1]:
# Ejemplo 1

from sklearn.datasets import load_breast_cancer 
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.feature_selection import SelectPercentile 
from sklearn.preprocessing import MinMaxScaler


# cancer is of type Bunch (Dictionary-like object)
cancer = load_breast_cancer ()
X = cancer['data']
y = cancer['target']



In [17]:
print("Número de características", X.shape)


Número de características (569, 30)


In [2]:
# Compute MI (mutual information) score

transformer = MinMaxScaler().fit(X)
X = transformer.transform(X)
#print( X[0:5] )

# Vector with mutual information scores
mi_score = mutual_info_classif(X,y)
print(mi_score)

# Since there are 30 characteristics, the algorithm generates 
# a vector with scores assigned to each one.

[0.36665582 0.09695648 0.40333271 0.35922432 0.08215737 0.21205567
 0.3733539  0.44121908 0.06436303 0.00651898 0.24536058 0.00134916
 0.27563103 0.339812   0.0146169  0.07563582 0.11845151 0.12691562
 0.01363194 0.04144752 0.45500854 0.12071894 0.47510987 0.4632881
 0.10723397 0.22553527 0.31515669 0.43781613 0.09553499 0.06840785]


In [3]:
# Set 1: train and test datasets (using all data). 
X_train_1,X_test_1,y_train_1,y_test_1 = train_test_split(X,y,random_state=0,stratify=y)

print("Datos de entrenamiento con todas las característica", X_train_1.shape)

Datos de entrenamiento con todas las característica (426, 30)


In [4]:
# Set 2: with features having MI scores > 0.2

# Characteristics with mi_score > 0.2
mi_score_selected_index = np.where(mi_score >0.2)[0]
print(mi_score_selected_index)

# X_2 data with features having MI scores > 0.2
X_2 = X[:,mi_score_selected_index]

X_train_2,X_test_2,y_train,y_test = train_test_split(X_2,y,random_state=0,stratify=y)

print("Datos de entrenamiento con características con peso de información mutua > 0.2", X_train_2.shape)

[ 0  2  3  5  6  7 10 12 13 20 22 23 25 26 27]
Datos de entrenamiento con características con peso de información mutua > 0.2 (426, 15)


In [5]:
# Dataset 3 with features have MI (mutual information) scores less than 0.2
mi_score_selected_index = np.where(mi_score <= 0.2)[0]
print(mi_score_selected_index)

X_3 = X[:,mi_score_selected_index]
X_train_3,X_test_3,y_train,y_test = train_test_split(X_3,y,random_state=0,stratify=y)

print("Datos de entrenamiento con características con peso de información mutua < 0.2", X_train_3.shape)

[ 1  4  8  9 11 14 15 16 17 18 19 21 24 28 29]
Datos de entrenamiento con características con peso de información mutua < 0.2 (426, 15)


In [7]:
# Test classifiers, one for each dataset with different columns.

from sklearn.tree import DecisionTreeClassifier 
model_1 = DecisionTreeClassifier().fit(X_train_1,y_train)
model_2 = DecisionTreeClassifier().fit(X_train_2,y_train)
model_3 = DecisionTreeClassifier().fit(X_train_3,y_train)

# Return the mean accuracy on the given test data and labels.
score_1 = model_1.score(X_test_1,y_test)
score_2 = model_2.score(X_test_2,y_test)
score_3 = model_3.score(X_test_3,y_test)
print(f"score_1:{score_1}\n score_2:{score_2}\n score_3:{score_3}")

score_1:0.9090909090909091
 score_2:0.916083916083916
 score_3:0.8321678321678322


### Selector de características de Scikit-Learn

Scikit-Learn proporciona selectores de características para no tener que calcular manualmente las puntuaciones de Información Mutua para seleccionar las características a utilizar. Por ejemplo, se puede seleccionar el 50% de características con mayor puntuación, otros selectores se usan de forma similar.

In [24]:
selector = SelectPercentile(percentile=50) # select features with top 50% 
selector.fit(X,y)
X_4 = selector.transform(X)

X_train_4,X_test_4,y_train,y_test = train_test_split(X_4,y,random_state=0,stratify=y)

model_4 = DecisionTreeClassifier().fit(X_train_4,y_train)
score_4 = model_4.score(X_test_4,y_test)

print(f"score_4:{score_4}")

score_4:0.9230769230769231


### Conclusión 

- El conjunto de datos 2 con 15 características con información mutua (MI) > 0.2 alcanza una precisión mayor que 0.92 tan buena como el conjunto de datos 1, que incluye todas las características. Mientras que score_3 es tiene una precisión de 0.83, que es el resultado de 15 características que tienen una puntuación MI <= 0.2.

- Es posible con el Método de información mutua, seleccionar las características que más aportan al ejercicio de clasificación en este conjunto de datos.

### Ejemplo 2  usando los datos de las especies de iris.


In [25]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler

irisData = load_iris()

# Cargado de datos 
X, y = load_iris(return_X_y=True)
print("Dimensiones del conjunto de datos",  X.shape )
print("Algunos registros", X[0:5] )

transformer = MinMaxScaler().fit(X)
X = transformer.transform(X)


X_new = mutual_info_classif(X, y, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)
print("Pesos de información mutua",  X_new )
# Nota: Este método requiere X e y, y analiza la información mutua entre cada uno de los
# atributos en X con y. print( X_new ) representa esa información, a más valor, más información
# aporta

Dimensiones del conjunto de datos (150, 4)
Algunos registros [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Pesos de información mutua [0.48889652 0.27416791 0.99603254 0.97979058]


In [10]:
print(irisData)

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

### Método de selección por Umbral de varianza (VarianceThreshold)

Selector de características que elimina todas las funciones de baja varianza.

Este algoritmo de selección analiza solo las características (X), no los resultados deseados (y), y por lo tanto puede usarse para el aprendizaje no supervisado.

In [26]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler

X, y = load_iris(return_X_y=True)
print( X.shape )
print( X[0:5] )

transformer = MinMaxScaler().fit(X)
X = transformer.transform(X)
print( X[0:5] )

selector = VarianceThreshold(threshold=.1)
X_new = selector.fit_transform(X)
print("varianzas: ", selector.variances_ )
print( X_new[0:5] )

# Este método analiza las varianzas en X, ojo que no usa y.  Y lo que imprime al final en
# selector.variances_ son las varianzas de cada atributo. En X_new quedan los atributos
# con varianzas mayores al umbral.


(150, 4)
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]
varianzas:  [0.05255573 0.03276265 0.08892567 0.10019668]
[[0.04166667]
 [0.04166667]
 [0.04166667]
 [0.04166667]
 [0.04166667]]


Es común utilizar medidas estadísticas de tipo correlación entre las variables de entrada y salida como base para la selección de características de filtro. Debido a esto, la elección de medidas estadísticas depende en gran medida de los tipos de datos de las variables. La siguiente figura (Brownlee, 2019) muestra algunos algoritmos estadísticos utilizados en el selección de característica. 

![](../imagenes/Metodos_de_filtro_estadisticos.png)




## Métodos de envoltura

Los métodos de envoltura **generan modelos con subconjuntos de características y miden los rendimientos de los modelos** construídos con tales subconjuntos. 

### Búsqueda hacia adelante

Este método evalúa en la primer etapa $n$ modelos construídos con las $n$ características específicas, para escoger el modelo con menor error con la característica $x_{i}$. En la segunda etapa, se combina tal característica $x_{i}$ con todas las restantes $n-1$ características para construír $n-1$ modelos, y escoger las mejores características $x_{i}$ y $x_{j}$. Tal secuencia de pasos se realiza de forma sucesiva para encontrar el conjunto de características más significativo, con un enfoque bottom-up.
![picture](../imagenes/Seleccion_caracteristicas_baa.png)


### Eliminación recursiva de características

La eliminación recursiva de características inicia con el modelo construído a partir de las $n$ características, y en la próxima iteración prueba los modelos al remover una característica, usando un enfoque top-down. 

![picture](../imagenes/Seleccion_caracteristicas_recElim.png)


## Métodos empotrados
Existen además los métodos empotrados los cuales echan mano de métodos de aprendizaje automático y regresión los cuales reducen y resaltan las dimensiones más importantes, como por ejemplo, la regresión LASSO, la cual se discutirá más adelante. 

## Ejemplos
En el siguiente link se aprecian varios métodos de selección de características en scikit-learn:

https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection

SelectKBest
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest

chi2
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2

Brownlee, J. (2019). How to Choose a Feature Selection Method For Machine Learning. Recuperado de https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/#:~:text=Feature%20selection%20is%20the%20process,the%20performance%20of%20the%20model.

Zhu, A. (2021). Select Features for Machine Learning Model with Mutual Information. Recuperado de: https://towardsdatascience.com/select-features-for-machine-learning-model-with-mutual-information-534fe387d5c8

Manual de Scikit-Learn v1.2.2 (n.d.). sklearn.feature_selection.mutual_info_classif. Recuperado de https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif

Manual de Scikit-Learn v1.2.2 (n.d.). sklearn.feature_selection.VarianceThreshold. Recuperado de https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold


In [12]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X, y = load_iris(return_X_y=True)
print( X.shape )
print( X[0:5] )
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
print( X_new.shape )
print( X_new[0:5] )

(150, 4)
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
(150, 2)
[[1.4 0.2]
 [1.4 0.2]
 [1.3 0.2]
 [1.5 0.2]
 [1.4 0.2]]
